<a href="https://colab.research.google.com/github/liz-lewis-manchester/CNM_2025_group_10/blob/Adam/CNM_Group_10_coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Group 10 Coding coursework

Completed by Adam Torossian, Joseph Haile, Martha Feasey and Fatin Binti Fairuz Rizal



#Main code and inputs to solve Test case 1

This has been ran with the parameters that satisfy the Test case 1 described in the brief, whereby a finite pulse is used.  

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Domain and Parameters

def define_domain(x_start=0.0, x_end=20.0, dx=0.2,
                  t_start=0.0, t_end=300.0, dt=10.0):
    #creates a spatial grid with spacing dx
    x = np.arange(x_start, x_end + dx, dx)
    #creates a time grid with spacing dt
    t = np.arange(t_start, t_end + dt, dt)
    #returns the grids for use in the solver
    return x, t, len(x), len(t)

def define_parameters(U=0.1):
  #return the flow velocity
    return U

# Initial Conditions
def get_initial_conditions_from_csv(csv_file='initial_conditions.csv', L=20.0, dx=0.2):
    try:
        #reads the csv file
        df = pd.read_csv(csv_file, encoding='latin1')
        #extracts the distance and concentration columns
        distance = df['Distance (m)'].values
        concentration = df['Concentration (µg/m_ )'].values
        #creates a model spatial grid
        x_grid = np.arange(0, L + dx, dx)
        #this interpolates the concentration values onto the model grid
        #this then allows the initial conditions to match the numerical domain
        return np.interp(x_grid, distance, concentration)
    except Exception as e:
      #handles errors like missing files or incorrect file names
        print(f"CSV read failed: {e}")
        return None

# Simulation Configuration

def configure_simulation(test_case_id, L=20.0, T=300.0, dx=0.2, dt=10.0, U=0.1):
  #number of spatial and temporal grid points
    Nx = int(L / dx) + 1
    Nt = int(T / dt) + 1
  #creates the temporal and spatial grid points
    x_grid = np.linspace(0, L, Nx)
    t_grid = np.linspace(0, T, Nt)
  #initialises the pollutant concentration array
    u_initial = np.zeros(Nx)

#test case 1 - Pulse release
    if test_case_id == 1:
#sets initial concentration at only x=0
        u_initial[0] = 250.0
#test case 2 - initial conditions from csv file
    elif test_case_id == 2:
        csv_data = get_initial_conditions_from_csv(L=L, dx=dx)
        if csv_data is not None and len(csv_data) == Nx:
            u_initial = csv_data
        else:
#fallback profile if the csv reading fails
            print("CSV failed, using fallback profile")
            csv_distance = [0, 5, 10, 15, 20]
            csv_concentration = [250, 50, 20, 10, 0]
            u_initial = np.interp(x_grid, csv_distance, csv_concentration)
#raise an error if the test case is invald
    else:
        raise ValueError(f"Test Case {test_case_id} not recognized")

#returns configurations for use in the solver
    return {
        'x': x_grid,
        't': t_grid,
        'u_init': u_initial,
        'velocity': U,
        'dx': dx,
        'dt': dt
    }

# Solver (recursive sweep)
def run_solver(config, decay_constant=0.0, random_perturbation=0.0,
               pulse_duration=3):
  #Implicit upwind solver for 1D advection with optional decay and optional velocity perturbation

  #extracts the necessary grids and parameters
  x, t = config['x'], config['t']
  U, dx, dt = config['velocity'], config['dx'], config['dt']
  Nt, Nx = len(t), len(x)

  #allocates an array to store concentration at all times and distances
  u = np.zeros((Nt, Nx))
  #sets initial concentration profile at t=0
  u[0, :] = config['u_init']

  for k in range(1, Nt):
      concentration_present = np.zeros(Nx)

      if k <= pulse_duration:
          concentration_present[0] = u[0, 0] * np.exp(-decay_constant * k * dt)
      else:
          concentration_present[0] = 0.0

      #velocity field with perturbation
      if random_perturbation > 0.0:
          # regenerate a new random profile each timestep
          random_variable = np.random.uniform(-1.0, 1.0, size=Nx)
          random_speed = U * (1.0 + (random_perturbation / 100.0) * random_variable)
      else:
          random_speed = np.full(Nx, U)

      # coefficients
      A_value = 1 / dt + random_speed / dx
      B_value = random_speed / dx

      #implicit upwind spatial sweep
      #concentration is updated sequentially from upstream to downstream using the previous timestep values
      for i in range(1, Nx):
          concentration_present[i] = (
              u[k - 1, i] / dt + B_value[i] * concentration_present[i - 1]
          ) / A_value[i]

      #apply decay everywhere (used for later test case)
      if decay_constant > 0.0:
          concentration_present *= np.exp(-decay_constant * dt)

      #store concentration for this timestep
      u[k, :] = concentration_present

  #return the full concentration field
  return u


# Animation
def animate_solution(x, t, u, interval=200):
#improves figure resolution
    plt.rcParams['figure.dpi'] = 150
#enable JavaScript-based animations (allows the graph to be seen in notebooks without having to call it forward)
    plt.rcParams["animation.html"] = "jshtml"
  #turns off the interactive plotting to avoid duplicate figures
    plt.ioff()

#create a figure with axes
    fig, ax = plt.subplots()

    def animate(frame):
#clears axes to redraw concentration profiles
        ax.cla()
#plots concentration along the river at current timestep
        ax.plot(x, u[frame, :])
#fits axis limits to ensure there is consistent scaling
        ax.set_xlim(0, x[-1])
        ax.set_ylim(0, np.max(u) * 1.1)
#labels the different axis
        ax.set_xlabel("Distance (m)")
        ax.set_ylabel("Concentration (μg/m³)")
        ax.set_title(f"Concentration at Time = {t[frame]:.1f} s")
#adds the grid to improve readability
        ax.grid(True)

#creates the animation by repeatedly calling the animation function
    anim = animation.FuncAnimation(fig, animate, frames=len(t), interval=interval)
    plt.close(fig) #closes the figure so it doesnt pile up

#returns the animation object (this allows it to be saved easier)
    return anim

# Main with Input Parameters
#allows user to choose between simple pulse inlet with input parameters or reading from the csv file
if __name__ == "__main__":
    test_case_id = int(input("Enter test case (1=pulse, 2=CSV): "))
    L = float(input("Enter river length (m): ")) #total length of the river
    T = float(input("Enter simulation time (s): ")) #total time for the simulation
    dx = float(input("Enter spatial resolution (m): ")) #the increment of distance on the graph
    dt = float(input("Enter time step (s): ")) #the increment of time on the graph
    U = float(input("Enter velocity (m/s): ")) #the flow velocity
    decay_constant = float(input("Enter decay constant (e.g., 0.0 for none): ")) #used for later test cases and adds pollutant decay
    perturbation_mode = float(input("Enter random perturbation % (0 for none): ")) #used for later test cases and adds random velocity variation

    config = configure_simulation(test_case_id, L=L, T=T, dx=dx, dt=dt, U=U) #configures the intial conditions and figure simulation based on selected test
    u = run_solver(config, decay_constant=decay_constant, random_perturbation=perturbation_mode) #runs the solver to compute the concentration of pollutant over space and time
    anim_result = animate_solution(config['x'], config['t'], u) #animates the results

#displays animation inline (so it can be seen in the notebook)
    from IPython.display import HTML
    display(HTML(anim_result.to_jshtml()))

Enter test case (1=pulse, 2=CSV): 1
Enter river length (m): 20
Enter simulation time (s): 300
Enter spatial resolution (m): 0.2
Enter time step (s): 10
Enter velocity (m/s): 0.1
Enter decay constant (e.g., 0.0 for none): 0
Enter random perturbation % (0 for none): 0


# Test case 2

Reads the initial conditons from the CSV file

In [11]:

# Test case 2 - Using the initial conditions in CSV file
test_case_id = 2        # 1 = pulse, 2 = CSV
decay_constant = 0.0    # pollutant decay constant
random_perturbation = 0.0  # % random velocity variation

# Run solver
config = configure_simulation(test_case_id, L=L, T=T, dx=dx, dt=dt, U=U)
u = run_solver(config, decay_constant=decay_constant, random_perturbation=random_perturbation)

# Animate results
anim_result = animate_solution(config['x'], config['t'], u)

from IPython.display import HTML
HTML(anim_result.to_jshtml())

import os

os.makedirs("results/test_case_2", exist_ok=True)

anim_result.save(
    "results/test_case_2/concentration_animation2.gif",
)



In [12]:
!ls results/test_case_2

concentration_animation2.gif


In [13]:
from google.colab import files
files.download("results/test_case_2/concentration_animation2.gif")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Test case 3

This test has been ran with multiple velocities to see how sensitive the solver is change in variables



In [29]:
import os
import matplotlib.pyplot as plt

# Create results folder
os.makedirs("results/test_case_3", exist_ok=True)

# Fixed parameters
test_case_id = 1
L = 20
T = 300
dx = 0.2
dt = 10

# Velocities to test
velocities = [0.05, 0.1, 0.2]

plt.figure()

for U in velocities:
    config = configure_simulation(
        test_case_id=test_case_id,
        L=L, T=T, dx=dx, dt=dt, U=U
    )
    u = run_solver(config)

    # Plot final time step
    plt.plot(config['x'], u[-1, :], label=f"U = {U} m/s")

plt.xlabel("Distance (m)")
plt.ylabel("Concentration (μg/m³)")
plt.title("Task 3: Sensitivity to flow velocity")
plt.legend()
plt.grid(True)

import os
import matplotlib.pyplot as plt

# Create results folder
os.makedirs("results/test_case_3", exist_ok=True)

# Fixed parameters
test_case_id = 1
L = 20
T = 300
dx = 0.2
dt = 10

# Velocities to test
velocities = [0.05, 0.1, 0.2]

plt.figure()

for U in velocities:
    config = configure_simulation(
        test_case_id=test_case_id,
        L=L, T=T, dx=dx, dt=dt, U=U
    )
    u = run_solver(config)

    # Plot final time step
    plt.plot(config['x'], u[-1, :], label=f"U = {U} m/s")

plt.xlabel("Distance (m)")
plt.ylabel("Concentration (μg/m³)")
plt.title("Task 3: Sensitivity to flow velocity")
plt.legend()
plt.grid(True)

plt.savefig("results/test_case_3/velocity_sensitivity.png")
plt.close()


In [ ]:
import os
import matplotlib.pyplot as plt

# Create results folder
os.makedirs("results/test_case_3", exist_ok=True)

# Fixed parameters
test_case_id = 1
L = 20
T = 300
dx = 0.2
dt = 10

# Velocities to test
velocities = [0.05, 0.1, 0.2]

plt.figure()

for U in velocities:
    config = configure_simulation(
        test_case_id=test_case_id,
        L=L, T=T, dx=dx, dt=dt, U=U
    )
    u = run_solver(config)

    # Plot final time step
    plt.plot(config['x'], u[-1, :], label=f"U = {U} m/s")

plt.xlabel("Distance (m)")
plt.ylabel("Concentration (μg/m³)")
plt.title("Task 3: Sensitivity to flow velocity")
plt.legend()
plt.grid(True)

plt.savefig("results/test_case_3/velocity_sensitivity.png")
plt.close()

In [30]:
!ls results/test_case_3

velocity_sensitivity.png


In [31]:
from google.colab import files
files.download("results/test_case_3/velocity_sensitivity.png")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Test case 4

Adding a decay constant

In [19]:
#Test case 4 - using a decay constant of 0.01
#this clearly flattens the curve much faster than in Test case 1 which occurs in the absence of a decay constant
test_case_id = 1        # 1 = pulse, 2 = CSV
L = 20.0                # river length (m)
T = 300.0               # simulation time (s)
dx = 0.2                # spatial resolution (m)
dt = 10.0               # time step (s)
U = 0.1                 # velocity (m/s)
decay_constant = 0.01    # pollutant decay constant
random_perturbation = 0.0  # % random velocity variation

# Run solver
config = configure_simulation(test_case_id, L=L, T=T, dx=dx, dt=dt, U=U)
u = run_solver(config, decay_constant=decay_constant, random_perturbation=random_perturbation)

# Animate results
anim_result = animate_solution(config['x'], config['t'], u)

from IPython.display import HTML
HTML(anim_result.to_jshtml())

import os

os.makedirs("results/test_case_4", exist_ok=True)

anim_result.save(
    "results/test_case_4/concentration_animation4.gif",
)

In [20]:
!ls results/test_case_4

concentration_animation3.gif  concentration_animation4.gif


In [21]:
from google.colab import files
files.download("results/test_case_4/concentration_animation4.gif")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Test case 5

Adding variability to the velocity

In [22]:
#Test case 5 - Adding variability to the velocity through perturbation
#Tested with a perturbation rate 0f 30% to make changes more visible
#The curve is more bumpy and presents a rougher shape through the process
#This is more obvious at higher perturbation percentages
test_case_id = 1        # 1 = pulse, 2 = CSV
L = 20.0                # river length (m)
T = 300.0               # simulation time (s)
dx = 0.2                # spatial resolution (m)
dt = 10.0               # time step (s)
U = 0.1                 # velocity (m/s)
decay_constant = 0.0    # pollutant decay constant
random_perturbation = 30  # % random velocity variation

# Run solver
config = configure_simulation(test_case_id, L=L, T=T, dx=dx, dt=dt, U=U)
u = run_solver(config, decay_constant=decay_constant, random_perturbation=random_perturbation)

# Animate results
anim_result = animate_solution(config['x'], config['t'], u)

from IPython.display import HTML
HTML(anim_result.to_jshtml())

import os

os.makedirs("results/test_case_5", exist_ok=True)

anim_result.save(
    "results/test_case_5/concentration_animation5.gif",
)

In [23]:
!ls results/test_case_5

concentration_animation5.gif


In [25]:
from google.colab import files
files.download("results/test_case_5/concentration_animation5.gif")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(*write your answer here. Double click to edit*)